In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

from estnltk.taggers import VabamorfCorpusTagger
vm_corpus_tagger = VabamorfCorpusTagger()

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [3]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

models = ['model_default',\
          'model_local_features_without_morph',\
          'model_morph_without_lemmas',\
          'model_morph_with_lemmas',\
          'model_morph_with_lemmas_and_sentences',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer',\
          'model_morph_with_lemmas_and_sentences_and_gazzetteer_and_global_features']

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [6]:
for model_directory in models:
    for subdistribution in [1, 2, 3, 4, 5]:
        training_subdistributions = []
        for y in [1, 2, 3, 4, 5]:
            if y == subdistribution:
                subdistribution_for_testing = y
            else:
                training_subdistributions.append(y)

        # Getting the filenames to be trained on from the files dictionary.
        filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}

        # Creating training_texts from the aforementioned filenames.
        print("Valmistan ette treenimistekste.")
        start = time.time()
        training_texts = []
        for filename in filenames:
            with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
                if filename in files_not_working:
                    continue
                else:
                    tagged_text = preprocess_text(json_to_text(file.read()))
                    tagged_text.pop_layer('morph_analysis')
                    training_texts.append(tagged_text)
        vm_corpus_tagger.tag(training_texts)
        print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
        # Setting up the trainer and training.
        print("Alustan NerTaggeri treenimist.")
        start = time.time()
        new_model_dir = os.path.join('models', model_directory)
        modelUtil = ModelStorageUtil( new_model_dir )
        nersettings = modelUtil.load_settings()
        trainer = NerTrainer(nersettings)
        trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
        print(f"NerTagger treenitud {time.time() - start} sekundiga.")

        # Setting up the new trained nertagger and defining layers to be removed later on.
        nertagger = NerTagger(model_dir = new_model_dir)
        removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

        # Tagging the files using the new nertagger.
        print("\n\nAlustan failide märgendamist.")
        start = time.time()
        testing_files = {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}
        for file in testing_files:
            with open(find(file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
                text = f.read()
                if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                    text = text.replace('..', '. .')
                text = preprocess_text(Text(text))
                text.pop_layer('morph_analysis')
                text = [text]
                vm_corpus_tagger.tag( text )
                nertagger.tag(text[0])
                text[0].add_layer(flatten(text[0]['ner'], 'flat_ner'))

                for x in removed_layers:
                    text[0].pop_layer(x)

                text_to_json(text[0], file=os.path.join(new_model_dir, 'vallakohtufailid-trained-nertagger', file))
                print(f'Märgendatud fail {file}')
        print(f"Failid märgendatud {time.time() - start} sekundiga.")
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 221.96380996704102 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 340748
Seconds required: 1.421

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30297.406772
Trial #1 (eta = 0.100000): 2603.262755
Trial #2 (eta = 0.200000): 3934.558978
Trial #3 (eta = 0.400000): 7760.285377
Trial #4 (eta = 0.800000): 16299.659386
Trial #5 (eta = 1.600000): 32054.750574 (worse)
Trial #6 (eta = 0.050000): 2338.748856
Tria

***** Epoch #36 *****
Loss: 483.437431
Improvement ratio: 0.383196
Feature L2-norm: 105.654499
Learning rate (eta): 0.049821
Total number of feature updates: 584748
Seconds required for this iteration: 0.524

***** Epoch #37 *****
Loss: 473.673042
Improvement ratio: 0.328099
Feature L2-norm: 106.312603
Learning rate (eta): 0.049816
Total number of feature updates: 600991
Seconds required for this iteration: 0.529

***** Epoch #38 *****
Loss: 444.838469
Improvement ratio: 0.367286
Feature L2-norm: 106.958633
Learning rate (eta): 0.049811
Total number of feature updates: 617234
Seconds required for this iteration: 0.524

***** Epoch #39 *****
Loss: 455.534210
Improvement ratio: 0.278841
Feature L2-norm: 107.589021
Learning rate (eta): 0.049806
Total number of feature updates: 633477
Seconds required for this iteration: 0.527

***** Epoch #40 *****
Loss: 438.670254
Improvement ratio: 0.290476
Feature L2-norm: 108.209618
Learning rate (eta): 0.049801
Total number of feature updates: 649720

***** Epoch #76 *****
Loss: 260.827970
Improvement ratio: 0.162393
Feature L2-norm: 123.737212
Learning rate (eta): 0.049623
Total number of feature updates: 1234468
Seconds required for this iteration: 0.529

***** Epoch #77 *****
Loss: 258.076923
Improvement ratio: 0.069183
Feature L2-norm: 124.058691
Learning rate (eta): 0.049618
Total number of feature updates: 1250711
Seconds required for this iteration: 0.506

***** Epoch #78 *****
Loss: 264.475863
Improvement ratio: 0.136060
Feature L2-norm: 124.369815
Learning rate (eta): 0.049613
Total number of feature updates: 1266954
Seconds required for this iteration: 0.686

***** Epoch #79 *****
Loss: 262.228167
Improvement ratio: 0.098209
Feature L2-norm: 124.672038
Learning rate (eta): 0.049608
Total number of feature updates: 1283197
Seconds required for this iteration: 0.771

***** Epoch #80 *****
Loss: 263.746720
Improvement ratio: 0.009636
Feature L2-norm: 124.970447
Learning rate (eta): 0.049603
Total number of feature updates: 12

Märgendatud fail Tartu_Laiuse_Kivij2rve_id13162_1866a.json
Märgendatud fail Tartu_V6nnu_Ahja_id20418_1888a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24521_1888a.json
Märgendatud fail P2rnu_Mihkli_Mihkli_id1099_1852a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16184_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id10343_1871a.json
Märgendatud fail L22ne_Ridala_V6nnu_id2373_1889a.json
Märgendatud fail Harju_Hageri_Kohila_id2634_1882a.json
Märgendatud fail L22ne_Martna_Martna_id10803_1890a.json
Märgendatud fail V6ru_Vastseliina_Misso_id13294_1887a.json
Märgendatud fail P2rnu_Tori_Sindi_id3778_1884a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16117_1883a.json
Märgendatud fail V6ru_Vastseliina_Misso_id13682_1886a.json
Märgendatud fail Tartu_V6nnu_Ahja_id3502_1882a.json
Märgendatud fail J2rva_Tyri_V22tsa_id22541_1914a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id24804_1877a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id10108_1880a.json
Märgendatud fail Tartu_V6nnu_Ahja_id13562_1881a.json
Märge

Märgendatud fail Tartu_Kodavere_Alatskivi_id11390_1880a.json
Märgendatud fail V6ru_Vastseliina_Misso_id11633_1886a.json
Märgendatud fail Viljandi_P6ltsamaa_Adavere_id20850_1896a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id3008_1884a.json
Märgendatud fail J2rva_Ambla_Ambla_id7255_1886a.json
Märgendatud fail Tartu_Torma_Avinurme_id23576_1872a.json
Märgendatud fail Harju_J6el2htme_J6el2htme_id7375_1869a.json
Märgendatud fail Tartu_Kodavere_Pala_id18163_1862a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id24037_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id22714_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16433_1884a.json
Märgendatud fail J2rva_Tyri_S2revere_id7087_1884a.json
Märgendatud fail J2rva_Tyri_S2revere_id9048_1880a.json
Märgendatud fail Harju_Keila_Saue_id14410_1886a.json
Märgendatud fail Saare_Kihelkonna_Pidula_id5660_1885a.json
Märgendatud fail Tartu_Kodavere_Pala_id21953_1870a.json
Märgendatud fail Tartu_Kodavere_Pala_id21737_1870a.json
Märgendatud fail Viljandi_Pilistvere

...

Märgendatud fail L22ne_Vormsi_Vormsi_id14921_1888a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id25012_1888a.json
Märgendatud fail J2rva_Tyri_V22tsa_id19025_1898a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16384_1885a.json
Märgendatud fail Harju_Kose_Palvere_id21108_1885a.json
Märgendatud fail Tartu_R6ngu_Aakre_id4809_1889a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id13022_1876a.json
Märgendatud fail Tartu_R6ngu_Aakre_id8898_1827a.json
Märgendatud fail Tartu_V6nnu_Ahja_id22170_1868a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id22089_1886a.json
Märgendatud fail J2rva_Anna_Purdi_id25497_1870a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17890_1885a.json
Märgendatud fail Viljandi_P6ltsamaa_Pajusi_id6831_1871a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23188_1872a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id1584_1878a.json
Märgendatud fail Viljandi_P6ltsamaa_Uue-P6ltsamaa_id6410_1853a.json
Märgendatud fail L22ne_Martna_Martna_id17524_1871a.json
Märgendatud fail V6ru_Vastseliina_Mis

***** Epoch #9 *****
Loss: 2011.298120
Feature L2-norm: 72.451585
Learning rate (eta): 0.049955
Total number of feature updates: 140526
Seconds required for this iteration: 0.495

***** Epoch #10 *****
Loss: 1792.917955
Feature L2-norm: 74.843091
Learning rate (eta): 0.049950
Total number of feature updates: 156140
Seconds required for this iteration: 0.494

***** Epoch #11 *****
Loss: 1594.856677
Improvement ratio: 10.872892
Feature L2-norm: 77.023510
Learning rate (eta): 0.049945
Total number of feature updates: 171754
Seconds required for this iteration: 0.495

***** Epoch #12 *****
Loss: 1440.130634
Improvement ratio: 5.028566
Feature L2-norm: 79.033342
Learning rate (eta): 0.049940
Total number of feature updates: 187368
Seconds required for this iteration: 0.514

***** Epoch #13 *****
Loss: 1315.687838
Improvement ratio: 3.862143
Feature L2-norm: 80.885907
Learning rate (eta): 0.049935
Total number of feature updates: 202982
Seconds required for this iteration: 0.494

***** Epoch

***** Epoch #49 *****
Loss: 375.037168
Improvement ratio: 0.184125
Feature L2-norm: 112.592933
Learning rate (eta): 0.049756
Total number of feature updates: 765086
Seconds required for this iteration: 0.493

***** Epoch #50 *****
Loss: 365.270190
Improvement ratio: 0.205238
Feature L2-norm: 113.080039
Learning rate (eta): 0.049751
Total number of feature updates: 780700
Seconds required for this iteration: 0.493

***** Epoch #51 *****
Loss: 355.872718
Improvement ratio: 0.229665
Feature L2-norm: 113.560401
Learning rate (eta): 0.049746
Total number of feature updates: 796314
Seconds required for this iteration: 0.494

***** Epoch #52 *****
Loss: 362.679787
Improvement ratio: 0.157233
Feature L2-norm: 114.033888
Learning rate (eta): 0.049741
Total number of feature updates: 811928
Seconds required for this iteration: 0.493

***** Epoch #53 *****
Loss: 358.526827
Improvement ratio: 0.155929
Feature L2-norm: 114.488611
Learning rate (eta): 0.049736
Total number of feature updates: 827542

***** Epoch #90 *****
Loss: 244.944181
Improvement ratio: 0.067328
Feature L2-norm: 127.194598
Learning rate (eta): 0.049554
Total number of feature updates: 1405260
Seconds required for this iteration: 0.492

***** Epoch #91 *****
Loss: 241.662079
Improvement ratio: 0.034192
Feature L2-norm: 127.456534
Learning rate (eta): 0.049549
Total number of feature updates: 1420874
Seconds required for this iteration: 0.496

***** Epoch #92 *****
Loss: 227.559231
Improvement ratio: 0.204048
Feature L2-norm: 127.720971
Learning rate (eta): 0.049544
Total number of feature updates: 1436488
Seconds required for this iteration: 0.493

***** Epoch #93 *****
Loss: 253.103812
Improvement ratio: -0.039004
Feature L2-norm: 127.981582
Learning rate (eta): 0.049539
Total number of feature updates: 1452102
Seconds required for this iteration: 0.494

SGD terminated with the stopping criteria
Loss: 227.559231
Total seconds required for training: 46.423

Storing the model
Number of active features: 339008 (33

Märgendatud fail P2rnu_Tori_Sindi_id7721_1885a.json
Märgendatud fail Tartu_V6nnu_Ahja_id16313_1883a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17919_1885a.json
Märgendatud fail J2rva_Tyri_V22tsa_id18360_1888a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17535_1885a.json
Märgendatud fail Harju_Rapla_Rapla_id17967_1868a.json
Märgendatud fail Harju_Hageri_Kohila_id20604_1883a.json
Märgendatud fail J2rva_Tyri_V22tsa_id22542_1915a.json
Märgendatud fail Tartu_V6nnu_Ahja_id23913_1898a.json
Märgendatud fail J2rva_Tyri_S2revere_id6037_1882a.json
Märgendatud fail V6ru_R6uge_Saaluse_id9257_1878a.json
Märgendatud fail Harju_Kose_Kose-Uuem6isa_id3179_1867a.json
Märgendatud fail J2rva_Anna_Purdi_id23132_1870a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21074_1889a.json
Märgendatud fail L22ne_Emmaste_Emmaste_id17875_1898a.json
Märgendatud fail Tartu_V6nnu_Ahja_id18479_1886a.json
Märgendatud fail P2rnu_Audru_V6lla_id5156_1877a.json
Märgendatud fail Viljandi_Paistu_Holstre_id10727_1905a.json
Märgendatud fail J2

In [7]:
for model_directory in models:
    extract_results_to_txt_file(model_directory, files)

Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
Tulemuste ammutamine on lõpetatud.
